In [2]:
import pandas as pd
from sklearn.linear_model import LinearRegression, SGDRegressor, Ridge, Lasso
from sklearn.metrics import max_error, mean_absolute_error, r2_score, mean_squared_error
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from joblib import dump
from sklearn.tree import DecisionTreeRegressor as DTR
import statsmodels.api as sm

In [14]:
df = pd.read_csv('../data/final/final_game_features_ps.csv')

In [15]:
df

,GAME_DATE_EST,GAME_ID,team_id,oppo_id,year,month,day,pointspread,loc,PTS_team,...,team_lag6_locps,team_lag7_locps,team_lag8_locps,team_lag9_locps,team_lag10_locps,team_lag11_locps,team_lag12_locps,team_lag13_locps,vs_oppo1_locps,vs_oppo2_locps
0,4/10/2022,22101229,1610612758,1610612756,2022,4,10,7,0,116,...,0,0,0,-3,-29,-9,9,0,-3,-5
1,4/9/2022,22101212,1610612758,1610612746,2022,4,9,-19,0,98,...,0,0,-3,-29,-9,9,0,-6,-16,5
2,4/5/2022,22101188,1610612758,1610612740,2022,4,5,-14,1,109,...,0,-3,-29,-9,9,0,-6,-16,0,13
3,4/3/2022,22101176,1610612758,1610612744,2022,4,3,-19,1,90,...,-3,-29,-9,9,0,-6,-16,0,0,0
4,4/1/2022,22101154,1610612758,1610612745,2022,4,1,5,0,122,...,-29,-9,9,0,-6,-16,0,0,0,-6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46253,11/3/2004,20400007,1610612738,1610612755,2004,11,3,-3,1,95,...,-7,0,0,0,0,5,-7,0,24,0
46254,4/25/2004,40300104,1610612738,1610612754,2004,4,25,-15,1,75,...,0,0,0,0,5,-7,0,24,-23,0
46255,4/23/2004,40300103,1610612738,1610612754,2004,4,23,-23,1,85,...,0,0,0,5,-7,0,24,9,0,0
46256,4/20/2004,40300102,1610612738,1610612754,2004,4,20,-13,0,90,...,0,0,5,-7,0,24,9,0,0,-18


In [16]:
y = df['pointspread']
X = df.iloc[:, 53:]
X['loc'] = df['loc']
X

,oppo_let_avg_pointspread,oppo_let_avg_PTS,oppo_let_avg_FG,oppo_let_avg_FT,oppo_let_avg_FG3,oppo_let_avg_AST,oppo_let_avg_REB,oppo_let_avg_FGM,oppo_let_avg_FGA,oppo_let_avg_FG3M,...,team_lag7_locps,team_lag8_locps,team_lag9_locps,team_lag10_locps,team_lag11_locps,team_lag12_locps,team_lag13_locps,vs_oppo1_locps,vs_oppo2_locps,loc
0,-10.146341,105.560976,0.433902,0.774561,0.335098,22.390244,43.536585,38.560976,89.024390,11.756098,...,0,0,-3,-29,-9,9,0,-3,-5,0
1,-0.100000,108.625000,0.452425,0.790200,0.334475,24.050000,46.150000,40.750000,90.225000,11.550000,...,0,-3,-29,-9,9,0,-6,-16,5,0
2,4.571429,110.619048,0.470119,0.803524,0.356714,25.452381,42.928571,40.380952,85.952381,13.000000,...,-3,-29,-9,9,0,-6,-16,0,13,1
3,-0.384615,108.410256,0.457205,0.739077,0.354410,24.051282,42.512821,39.358974,86.000000,12.948718,...,-29,-9,9,0,-6,-16,0,0,0,1
4,6.300000,117.725000,0.474775,0.774250,0.344825,24.875000,46.600000,43.100000,90.800000,12.800000,...,-9,9,0,-6,-16,0,0,0,-6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46253,5.853659,93.390244,0.443561,0.744854,0.323415,21.951220,42.439024,34.926829,79.000000,5.219512,...,0,0,0,0,5,-7,0,24,0,1
46254,-3.025000,88.625000,0.441950,0.756100,0.338650,20.425000,40.150000,33.675000,76.500000,4.000000,...,0,0,0,5,-7,0,24,-23,0,1
46255,-2.550000,88.975000,0.443125,0.753450,0.343625,20.500000,40.000000,33.775000,76.525000,4.075000,...,0,0,5,-7,0,24,9,0,0,1
46256,-9.142857,82.452381,0.423571,0.747643,0.304762,19.738095,40.190476,31.380952,74.095238,4.095238,...,0,5,-7,0,24,9,0,0,-18,0


In [17]:
X = sm.add_constant(X)
m = sm.OLS(y, X).fit()
print(m.summary())

                            OLS Regression Results                            
Dep. Variable:            pointspread   R-squared:                       0.222
Model:                            OLS   Adj. R-squared:                  0.220
Method:                 Least Squares   F-statistic:                     88.52
Date:                Sun, 01 Jan 2023   Prob (F-statistic):               0.00
Time:                        12:51:06   Log-Likelihood:            -1.8138e+05
No. Observations:               46258   AIC:                         3.631e+05
Df Residuals:                   46108   BIC:                         3.644e+05
Df Model:                         149                                         
Covariance Type:            nonrobust                                         
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
const             

In [18]:
clf = LinearRegression()
clf.fit(X, y)
y_true, y_pred = y, clf.predict(X)
print('rooted_mean_squared_error:' + str(mean_squared_error(y_true, y_pred, squared=False)))
print('max_error:' + str(max_error(y_true, y_pred)))
print('mean_absolute_error:' + str(mean_absolute_error(y_true, y_pred)))
print('r2_score:' + str(r2_score(y_true, y_pred)))

rooted_mean_squared_error:12.209550000118172
max_error:63.70288476233913
mean_absolute_error:9.57534311585327
r2_score:0.22243293390232377


In [3]:
df_ps = pd.read_csv('../data/final/final_game_features_ps.csv')
df_pts = pd.read_csv('../data/final/final_game_features_pts.csv')
df_ps

,GAME_DATE_EST,GAME_ID,team_id,oppo_id,year,month,day,pointspread,loc,PTS_team,...,vs_oppo2_FG3,vs_oppo2_FTM,vs_oppo2_FTA,vs_oppo2_PLUS,vs_oppo2_PPS,vs_oppo2_FIC,team_PTS_pred,team_PMS_pred,oppo_PTS_pred,oppo_PMS_pred
0,4/10/2022,22101229,1610612758,1610612756,2022,4,10,7,0,116,...,0.417,20,26,-25,1.024390,66.250,98.754286,-57.314832,86.764515,28.438087
1,4/9/2022,22101212,1610612758,1610612746,2022,4,9,-19,0,98,...,0.353,20,25,25,0.976744,78.625,102.313099,-27.096538,144.706594,20.058562
2,4/5/2022,22101188,1610612758,1610612740,2022,4,5,-14,1,109,...,0.333,18,22,65,1.175000,96.000,102.579057,-5.217212,116.545639,12.852550
3,4/3/2022,22101176,1610612758,1610612744,2022,4,3,-19,1,90,...,0.406,11,16,-75,1.048193,71.500,103.909598,-13.424426,95.684678,5.807290
4,4/1/2022,22101154,1610612758,1610612745,2022,4,1,5,0,122,...,0.316,18,24,-30,1.032967,84.250,109.180691,-2.874840,82.567293,1.601809
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46253,11/3/2004,20400007,1610612738,1610612755,2004,11,3,-3,1,95,...,0.474,25,37,150,1.180556,81.125,84.154036,7.462727,98.482950,-8.619179
46254,4/25/2004,40300104,1610612738,1610612754,2004,4,25,-15,1,75,...,0.217,31,38,-65,0.880597,58.750,91.097657,-19.337818,91.095096,29.981732
46255,4/23/2004,40300103,1610612738,1610612754,2004,4,23,-23,1,85,...,0.363,22,29,-80,0.916667,51.625,90.590308,-13.650073,101.373520,25.366713
46256,4/20/2004,40300102,1610612738,1610612754,2004,4,20,-13,0,90,...,0.267,15,18,-90,0.868421,42.250,87.605853,-38.144594,78.661305,37.304328


In [4]:
y_ps = df_ps['pointspread']
X_ps = df_ps.iloc[:, 53:]
X_ps['loc'] = df_ps['loc']
X_ps

,oppo_let_avg_pointspread,oppo_let_avg_FG,oppo_let_avg_FT,oppo_let_avg_FG3,oppo_let_avg_AST,oppo_let_avg_REB,oppo_let_avg_FTM,oppo_let_avg_FTA,oppo_let_avg_OREB,oppo_let_avg_DREB,...,vs_oppo2_FTM,vs_oppo2_FTA,vs_oppo2_PLUS,vs_oppo2_PPS,vs_oppo2_FIC,team_PTS_pred,team_PMS_pred,oppo_PTS_pred,oppo_PMS_pred,loc
0,-10.146341,0.433902,0.774561,0.335098,22.390244,43.536585,16.682927,21.634146,10.560976,32.975610,...,20,26,-25,1.024390,66.250,98.754286,-57.314832,86.764515,28.438087,0
1,-0.100000,0.452425,0.790200,0.334475,24.050000,46.150000,15.575000,19.750000,11.600000,34.525000,...,20,25,25,0.976744,78.625,102.313099,-27.096538,144.706594,20.058562,0
2,4.571429,0.470119,0.803524,0.356714,25.452381,42.928571,16.857143,21.071429,8.785714,34.142857,...,18,22,65,1.175000,96.000,102.579057,-5.217212,116.545639,12.852550,1
3,-0.384615,0.457205,0.739077,0.354410,24.051282,42.512821,16.743590,22.410256,9.051282,33.461538,...,11,16,-75,1.048193,71.500,103.909598,-13.424426,95.684678,5.807290,1
4,6.300000,0.474775,0.774250,0.344825,24.875000,46.600000,18.725000,24.400000,11.850000,34.750000,...,18,24,-30,1.032967,84.250,109.180691,-2.874840,82.567293,1.601809,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46253,5.853659,0.443561,0.744854,0.323415,21.951220,42.439024,18.317073,24.634146,12.780488,29.658537,...,25,37,150,1.180556,81.125,84.154036,7.462727,98.482950,-8.619179,1
46254,-3.025000,0.441950,0.756100,0.338650,20.425000,40.150000,17.275000,22.875000,10.775000,29.375000,...,31,38,-65,0.880597,58.750,91.097657,-19.337818,91.095096,29.981732,1
46255,-2.550000,0.443125,0.753450,0.343625,20.500000,40.000000,17.350000,23.075000,10.625000,29.375000,...,22,29,-80,0.916667,51.625,90.590308,-13.650073,101.373520,25.366713,1
46256,-9.142857,0.423571,0.747643,0.304762,19.738095,40.190476,15.595238,20.833333,10.428571,29.761905,...,15,18,-90,0.868421,42.250,87.605853,-38.144594,78.661305,37.304328,0


In [5]:
X_ps = sm.add_constant(X_ps)
m = sm.OLS(y_ps, X_ps).fit()
print(m.summary())

                            OLS Regression Results                            
Dep. Variable:            pointspread   R-squared:                       0.226
Model:                            OLS   Adj. R-squared:                  0.220
Method:                 Least Squares   F-statistic:                     38.31
Date:                Wed, 04 Jan 2023   Prob (F-statistic):               0.00
Time:                        10:10:04   Log-Likelihood:            -1.8128e+05
No. Observations:               46258   AIC:                         3.633e+05
Df Residuals:                   45907   BIC:                         3.663e+05
Df Model:                         350                                         
Covariance Type:            nonrobust                                         
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
const             

In [6]:
clf = LinearRegression()
clf.fit(X_ps, y_ps)
y_true, y_pred = y_ps, clf.predict(X_ps)
print('rooted_mean_squared_error:' + str(mean_squared_error(y_true, y_pred, squared=False)))
print('max_error:' + str(max_error(y_true, y_pred)))
print('mean_absolute_error:' + str(mean_absolute_error(y_true, y_pred)))
print('r2_score:' + str(r2_score(y_true, y_pred)))

rooted_mean_squared_error:12.181042264225185
max_error:62.44435706749437
mean_absolute_error:9.554364406751002
r2_score:0.22605973397004986


In [7]:
y_pts = df_pts['PTS_team']
X_pts = df_pts.iloc[:, 53:]
X_pts['loc'] = df_pts['loc']
X_pts

,oppo_let_avg_PTS,oppo_let_avg_FG,oppo_let_avg_FT,oppo_let_avg_FG3,oppo_let_avg_AST,oppo_let_avg_REB,oppo_let_avg_FGM,oppo_let_avg_FGA,oppo_let_avg_FG3A,oppo_let_avg_FTM,...,vs_oppo2_FT,vs_oppo2_FG3,vs_oppo2_REB,vs_oppo2_FGA,vs_oppo2_FIC,team_PTS_pred,team_PMS_pred,oppo_PTS_pred,oppo_PMS_pred,loc
0,105.560976,0.433902,0.774561,0.335098,22.390244,43.536585,38.560976,89.024390,34.951220,16.682927,...,0.769,0.417,46,82,66.250,98.754286,-57.314832,86.764515,28.438087,0
1,108.625000,0.452425,0.790200,0.334475,24.050000,46.150000,40.750000,90.225000,34.125000,15.575000,...,0.800,0.353,51,86,78.625,102.313099,-27.096538,144.706594,20.058562,0
2,110.619048,0.470119,0.803524,0.356714,25.452381,42.928571,40.380952,85.952381,36.309524,16.857143,...,0.818,0.333,37,80,96.000,102.579057,-5.217212,116.545639,12.852550,1
3,108.410256,0.457205,0.739077,0.354410,24.051282,42.512821,39.358974,86.000000,36.564103,16.743590,...,0.688,0.406,37,83,71.500,103.909598,-13.424426,95.684678,5.807290,1
4,117.725000,0.474775,0.774250,0.344825,24.875000,46.600000,43.100000,90.800000,36.925000,18.725000,...,0.750,0.316,37,91,84.250,109.180691,-2.874840,82.567293,1.601809,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46253,93.390244,0.443561,0.744854,0.323415,21.951220,42.439024,34.926829,79.000000,16.243902,18.317073,...,0.676,0.474,40,72,81.125,84.154036,7.462727,98.482950,-8.619179,1
46254,88.625000,0.441950,0.756100,0.338650,20.425000,40.150000,33.675000,76.500000,12.250000,17.275000,...,0.816,0.217,38,67,58.750,91.097657,-19.337818,91.095096,29.981732,1
46255,88.975000,0.443125,0.753450,0.343625,20.500000,40.000000,33.775000,76.525000,12.300000,17.350000,...,0.759,0.363,38,72,51.625,90.590308,-13.650073,101.373520,25.366713,1
46256,82.452381,0.423571,0.747643,0.304762,19.738095,40.190476,31.380952,74.095238,12.619048,15.595238,...,0.833,0.267,38,76,42.250,87.605853,-38.144594,78.661305,37.304328,0


In [8]:
X_pts = sm.add_constant(X_pts)
m = sm.OLS(y_pts, X_pts).fit()
print(m.summary())

                            OLS Regression Results                            
Dep. Variable:               PTS_team   R-squared:                       0.322
Model:                            OLS   Adj. R-squared:                  0.318
Method:                 Least Squares   F-statistic:                     82.69
Date:                Wed, 04 Jan 2023   Prob (F-statistic):               0.00
Time:                        10:13:00   Log-Likelihood:            -1.7632e+05
No. Observations:               46258   AIC:                         3.532e+05
Df Residuals:                   45993   BIC:                         3.555e+05
Df Model:                         264                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                  -30.8677 

In [9]:
clf = LinearRegression()
clf.fit(X_pts, y_pts)
y_true, y_pred = y_pts, clf.predict(X_pts)
print('rooted_mean_squared_error:' + str(mean_squared_error(y_true, y_pred, squared=False)))
print('max_error:' + str(max_error(y_true, y_pred)))
print('mean_absolute_error:' + str(mean_absolute_error(y_true, y_pred)))
print('r2_score:' + str(r2_score(y_true, y_pred)))

rooted_mean_squared_error:10.943449168058962
max_error:57.14583975003289
mean_absolute_error:8.657813466003766
r2_score:0.32186540286036425
